## Referencce website 
code from https://blog.csdn.net/dgvv4/article/details/122023047

### **Import library**

In [1]:
import cv2

In [2]:
import mediapipe as mp

In [3]:
import time

In [4]:
import csv
import os

In [5]:
import sqlite3

In [6]:
from datetime import datetime

### **Create database**

In [ ]:
#conn = sqlite3.connect('hand_coordinates.db')

In [ ]:
# 创建一个表
#c = conn.cursor()
# 创建用于保存归一化坐标的表
#c.execute('''
#CREATE TABLE IF NOT EXISTS normalized_coordinates (
    #id INTEGER PRIMARY KEY AUTOINCREMENT,
    #timestamp TEXT,
    #x REAL,
    #y REAL
#);
#''')

# 创建用于保存像素坐标的表
#c.execute('''
#CREATE TABLE IF NOT EXISTS pixel_coordinates (
   # id INTEGER PRIMARY KEY AUTOINCREMENT,
   # timestamp TEXT,
   # x INTEGER,
   # y INTEGER
#);
#''')
#conn.commit()

### **Visualize hand landmarks and connections**

In [7]:
cap = cv2.VideoCapture(0)#capture video, 0 means built-in webcam

In [8]:
mpHands = mp.solutions.hands

In [9]:
hands = mpHands.Hands(static_image_mode=False, #静态追踪，低于0.5置信度会再一次跟踪
                      max_num_hands=2, # 最多有2只手
                      min_detection_confidence=0.5, # 最小检测置信度
                      min_tracking_confidence=0.5)  # 最小跟踪置信度

In [10]:
mpDraw = mp.solutions.drawing_utils

In [11]:
pTime = 0 #处理一张图像前的时间
cTime = 0 #一张图处理完的时间

In [12]:
# 存放坐标信息
lmList = []
captured_coordinates = []

In [28]:
#使用数据库保存得到的坐标
conn = sqlite3.connect('hand_coordinates.db')
# 创建一个表
db = conn.cursor()
# 创建用于保存归一化坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS normalized_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x REAL,
    y REAL
);
''')

# 创建用于保存像素坐标的表
db.execute('''
CREATE TABLE IF NOT EXISTS pixel_coordinates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    x INTEGER,
    y INTEGER
);
''')
conn.commit()
# 初始化
cap = cv2.VideoCapture(0)


normalized_coords = []
pixel_coords = []
lmList = []  
pTime = 0

# 创建保存数据的目录
output_dir = "output_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

while True:
    # 视频读取和预处理
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    print(results.multi_hand_landmarks)

    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            for index, lm in enumerate(handlms.landmark):
                if index == 8:  # 如果你只关心索引为8的关键点
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    print(index, cx, cy)
                    lmList.append([index, cx, cy])
                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

                    # 插入数据到数据库
                    db.execute("INSERT INTO normalized_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, lm.x, lm.y))

                    # 将像素坐标保存到数据库
                    db.execute("INSERT INTO pixel_coordinates (timestamp, x, y) VALUES (?, ?, ?)", (current_time, cx, cy))

                    conn.commit()

                    # 在图像上画圆
                    cv2.circle(img, (cx, cy), 12, (0, 220, 23), cv2.FILLED)

            # 其他代码（绘制关键点等）

    # FPS显示和其他后处理
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

    cv2.imshow('Image', img)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# 关闭数据库连接
conn.close()
# 释放摄像头资源
cap.release()
# 关闭所有 OpenCV 窗口
cv2.destroyAllWindows()

None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.9391307830810547
  y: 0.5741117596626282
  z: 2.006114812047599e-07
}
landmark {
  x: 0.8898868560791016
  y: 0.5480751991271973
  z: -0.01842525787651539
}
landmark {
  x: 0.8543031811714172
  y: 0.4867168664932251
  z: -0.03198853135108948
}
landmark {
  x: 0.8545714616775513
  y: 0.4179176092147827
  z: -0.04316352307796478
}
landmark {
  x: 0.8728867173194885
  y: 0.3659329116344452
  z: -0.0538431853055954
}
landmark {
  x: 0.8440491557121277
  y: 0.4394349157810211
  z: -0.03226182609796524
}
landmark {
  x: 0.8137331008911133
  y: 0.3540579378604889
  z: -0.058995191007852554
}
landmark {
  x: 0.7995797991752625
  y: 0.28960591554641724
  z: -0.08024302124977112
}
landmark {
  x: 0.7908828258514404
  y: 0.235524982213974
  z: -0.09534287452697754
}
landmark {
  x: 0.886507511138916
  y: 0.42260611057281494
  z: -0.03516149893403053
}
landmark {
  x: 0.8811802268028259
  y: 0.3344871699810028
  z: -0.0

In [20]:
#娴熟数据
conn = sqlite3.connect('hand_coordinates.db')
db = conn.cursor()
# 查询数据库中的数据
db.execute("SELECT * FROM normalized_coordinates")

# 获取查询结果
rows = db.fetchall()

# 打印查询结果
for row in rows:
    print(row)


(1, '2023-09-01 00:27:11.657272', 0.179504856467247, 0.6095626354217529)
(2, '2023-09-01 00:27:11.720410', 0.18962252140045166, 0.5748354196548462)
(3, '2023-09-01 00:27:11.782502', 0.2017063945531845, 0.5310595035552979)
(4, '2023-09-01 00:27:11.842065', 0.21591876447200775, 0.4869994819164276)
(5, '2023-09-01 00:27:11.906288', 0.26677411794662476, 0.29069820046424866)
(6, '2023-09-01 00:27:11.913000', 0.2480737864971161, 0.41227495670318604)
(7, '2023-09-01 00:27:11.954925', 0.26518499851226807, 0.2878594696521759)
(8, '2023-09-01 00:27:12.024757', 0.2737652659416199, 0.2395896017551422)
(9, '2023-09-01 00:27:12.078133', 0.3167172074317932, 0.15099439024925232)
(10, '2023-09-01 00:27:12.142713', 0.3350602388381958, 0.11227965354919434)
(11, '2023-09-01 00:27:12.205245', 0.35672688484191895, 0.09727948904037476)
(12, '2023-09-01 00:27:12.267080', 0.3719172477722168, 0.08844912052154541)
(13, '2023-09-01 00:27:12.330700', 0.410015344619751, 0.0687493085861206)
(14, '2023-09-01 00:27:12

In [29]:
#查看数据库
import pandas as pd

conn = sqlite3.connect('hand_coordinates.db')

# 将数据读入Pandas DataFrame
df = pd.read_sql_query("SELECT * from normalized_coordinates", conn)

# 显示数据
print(df)

        id                   timestamp         x         y
0        1  2023-09-01 00:27:11.657272  0.179505  0.609563
1        2  2023-09-01 00:27:11.720410  0.189623  0.574835
2        3  2023-09-01 00:27:11.782502  0.201706  0.531060
3        4  2023-09-01 00:27:11.842065  0.215919  0.486999
4        5  2023-09-01 00:27:11.906288  0.266774  0.290698
...    ...                         ...       ...       ...
3177  3178  2023-09-01 00:45:15.501034  0.352338  0.234602
3178  3179  2023-09-01 00:45:15.563086  0.345330  0.263423
3179  3180  2023-09-01 00:45:15.624085  0.337325  0.280784
3180  3181  2023-09-01 00:45:15.688121  0.310926  0.308413
3181  3182  2023-09-01 00:45:15.751212  0.304686  0.307964

[3182 rows x 4 columns]
